In [9]:
#Packages to install

# pretty printer
import pprint

# set up the pretty printer
pp = pprint.PrettyPrinter(indent=4)

# BeautifulSoup for scraping
from bs4 import BeautifulSoup

# for making HTTP requests
import requests

# Pandas/numpy for data manipulation
import pandas as pd
import numpy as np

In [10]:
# The URL for the Wikipedia page we're scraping
WIKI_URL = 'https://en.wikipedia.org/wiki/S%26P_100'

# Retrieve the page
wiki_page = requests.get(WIKI_URL).text

In [11]:
# parse the HTML text into a tree
soup = BeautifulSoup(wiki_page, 'html.parser')

# print the tree to screen
#print(soup.prettify())

In [12]:
# extract the table containin the S&P companies
sandp_table = soup.find('table', {"class" : "wikitable sortable"})

In [13]:
# snps array will hold an array of tuples of the form (Symbol, Name)
snps = []

# scan the table for each row ('tr' is the HTML tag for a table row)
for row in sandp_table.find_all('tr'):
    
    # scan the row for table cells ('td' is the tag for table data)
    cols = row.find_all('td')
    
    if len(cols) == 2: # skip the header row
        snps.append((cols[0].text.strip(), cols[1].text.strip()))

# convert the array of tuples into a Pandas DataFrame        
snps_df = pd.DataFrame(snps, columns=['Symbol', 'Name'])

snps_df.head()

,Symbol,Name
0,AAPL,Apple Inc.
1,ABBV,AbbVie Inc.
2,ABT,Abbott Laboratories
3,ACN,Accenture plc
4,AGN,Allergan plc


In [89]:
REUTERS_BASE_URL = 'http://www.reuters.com/finance/stocks/company-officers/'

symbol_array = snps_df['Symbol'].values

# board_members will hold an array of tuples, one for each board member
board_members = []

# for simplicity only look at the first five companies in class
for (index, co) in snps_df.iterrows():
    sym = co['Symbol']
    reuters_page = requests.get(REUTERS_BASE_URL+sym).text
    soup = BeautifulSoup(reuters_page, 'html.parser')
    company_news_table = soup.find(id="companyNews").find("tbody",{"class" : "dataSmall"})
    for row in company_news_table.find_all('tr'):
        cols = row.find_all('td')
        if(len(cols)==4):
            board_members.append((sym, cols[0].text.strip(),'https://www.reuters.com'+cols[0].find('a').get('href'), cols[1].text.strip(), cols[2].text.strip()))
    
df = pd.DataFrame(board_members, columns=['Symbol', 'Name','Link','Age', 'Year'])
df.head()


,Symbol,Name,Link,Age,Year
0,AAPL,Art Levinson,https://www.reuters.com/finance/stocks/officer...,67,2011
1,AAPL,Timothy Cook,https://www.reuters.com/finance/stocks/officer...,57,2011
2,AAPL,Luca Maestri,https://www.reuters.com/finance/stocks/officer...,54,2014
3,AAPL,Katherine Adams,https://www.reuters.com/finance/stocks/officer...,53,2017
4,AAPL,Phil Schiller,https://www.reuters.com/finance/stocks/officer...,57,2002


In [90]:
# Lab Problem 2
# Create a multiindex as described above

df.set_index(keys=['Symbol','Name'],inplace=True)
df.head()

Link Age  \
Symbol Name                                                                     
AAPL   Art Levinson     https://www.reuters.com/finance/stocks/officer...  67   
       Timothy Cook     https://www.reuters.com/finance/stocks/officer...  57   
       Luca Maestri     https://www.reuters.com/finance/stocks/officer...  54   
       Katherine Adams  https://www.reuters.com/finance/stocks/officer...  53   
       Phil Schiller    https://www.reuters.com/finance/stocks/officer...  57   

                        Year  
Symbol Name                   
AAPL   Art Levinson     2011  
       Timothy Cook     2011  
       Luca Maestri     2014  
       Katherine Adams  2017  
       Phil Schiller    2002

In [91]:
# Lab problem 3
#Use the indexed df to retrieve the entries for GOOG.

df.loc['GOOG']

,Link,Age,Year
Name,,,
John Hennessy,https://www.reuters.com/finance/stocks/officer...,65,2018
Sergey Brin,https://www.reuters.com/finance/stocks/officer...,44,2015
Lawrence Page,https://www.reuters.com/finance/stocks/officer...,45,2015
Ruth Porat,https://www.reuters.com/finance/stocks/officer...,60,2015
Sundar Pichai,https://www.reuters.com/finance/stocks/officer...,46,2017
David Drummond,https://www.reuters.com/finance/stocks/officer...,55,2015
Diane Greene,https://www.reuters.com/finance/stocks/officer...,62,2015
Eric Schmidt,https://www.reuters.com/finance/stocks/officer...,62,2018
L. John Doerr,https://www.reuters.com/finance/stocks/officer...,66,2016


In [92]:
# Lab problem 4
# compute and print the mean age of each board
df['Age'] = df['Age'].apply(pd.to_numeric, errors='coerce')
df.groupby('Symbol').mean().head()

,Age
Symbol,
AAPL,59.066667
ABBV,57.631579
ABT,59.550000
ACN,56.416667
AGN,57.470588


In [93]:
# Find the boards with the maximum and minimum average ages

new_df = df.groupby('Symbol').mean()
new_df[new_df.Age == new_df.Age.max()].index.item()
print("The boards with the maximum age: ", new_df[new_df.Age == new_df.Age.max()].index.item())
print("The boards with the minimum age: ", new_df[new_df.Age == new_df.Age.min()].index.item())

#print(df[df.Age == df.Age.max()]) 

The boards with the maximum age:  BRK.B
The boards with the minimum age:  KHC


## Homework Problem 1

Retrieve the total compensation of each board member and put the results in a DataFrame of the following columns:

Symbol: The company stock symbol.
Name: Board Member's name.
Total: Total yearly compensation for the member.
To do this problem you'll use the linkattribute in the df DataFrame from the lab.

You should decide how to index the DataFrame to best utilize it for subsequent problems.

In [94]:
def getPeopleCompensation(link):
    compensation_page = requests.get(link).text
    soup = BeautifulSoup(compensation_page, 'html.parser')
    compenstaionTable =soup.find_all('tr')[0]
    conpensation = None
    if 'Annual Compensation, USD' in compenstaionTable.text:
        allTD = compenstaionTable.find_all('td')
        sal_value = allTD[1].text.strip().replace(',', '')
        if sal_value.isdigit():
            conpensation = int(sal_value)
    return conpensation


In [95]:
df.head()
temp_df = df.reset_index()

In [98]:
board_members_compensation = []

for index, col in temp_df.iterrows():
    symbol = col['Symbol']
    name = col['Name']
    link = col['Link']
    conpensation = getPeopleCompensation(link)
    board_members_compensation.append((symbol, name, conpensation))

compensation_table = pd.DataFrame(board_members_compensation, columns=['Symbol', 'Name','Annual Compensation'])
compensation_table.head()

,Symbol,Name,Annual Compensation
0,AAPL,Art Levinson,300000.0
1,AAPL,Timothy Cook,12384700.0
2,AAPL,Luca Maestri,4128230.0
3,AAPL,Katherine Adams,NaN
4,AAPL,Phil Schiller,869942.0


# Homework Problem 2

Compute the mean compensation for each company and put the results in a DataFrame with the following columns:

Symbol: The company stock symbol
Compensation: Mean board compensation
Age: Mean board age
Notice that you're asked to include the mean age. This suggests that you will join two tables together. How will you index these two tables in order to compute the result elegantly and simply?

In [110]:
company_compansation_table = None

df_merge = pd.merge(temp_df, compensation_table, left_on=['Symbol', 'Name'], right_on=['Symbol', 'Name'])

company_compensation_table = df_merge.groupby('Symbol')['Annual Compensation', 'Age'].mean()

company_compensation_table.head(10)

,Annual Compensation,Age
Symbol,,
AAPL,2.570582e+06,59.066667
ABBV,1.316123e+06,57.631579
ABT,1.171487e+06,59.550000
ACN,1.055134e+06,56.416667
AGN,5.913817e+06,57.470588
AIG,1.912256e+06,61.041667
ALL,1.377166e+06,59.291667
AMGN,3.426171e+05,60.750000
AMZN,1.533680e+05,59.750000


# Homework Problem 3

You might have noticed that some people sit on more than one board. This is sometimes an indication of potential conflicts of interest. In this problem, you will identify members that on serve multiple boards.

The output will be a DataFrame with columns:

Name: Board member.
Symbol: Symbol of company
Count: Number of companies the board member serves
This problem requires you to revisit your Pandas aggregation chops. You're essentially taking df from before and applying a transformation. One or two lines of code if you leverage pandas; many lines of procedural code if you don't.

In [158]:
conflicts_table = None

conflicts_table = temp_df.groupby(['Name', 'Symbol']).Name.agg({ 'Count':'count'})

conflicts_table.head(30)

/Users/ayub/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


,,Count
Name,Symbol,
A. Brooke Seawell,NVDA,1
A. Eugene Washington,JNJ,1
A. Louise Pentland,PYPL,1
A. Robert Bailey,AGN,1
Aaron Abramovitz,SO,1
Aaron Floridia,SLB,1
Aaron Karczmer,PYPL,1
Aarti Shah,LLY,1
Abdellah Merad,SLB,1
